<a href="https://colab.research.google.com/github/kyle1130/README.md/blob/main/4_llm%EC%9C%BC%EB%A1%9C_%ED%95%99%EC%8A%B5_dataset_%EB%A7%8C%EB%93%A4%EA%B8%B0(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

라이브러리 설치

In [1]:
!pip install bertopic
!pip install -q transformers bertopic timesfm
!pip install -q scikit-learn

# -*- coding: utf-8 -*-
from google.colab import drive

import io
import os
import json

import torch

import numpy as np
import re
from datetime import datetime
from tqdm import tqdm
from collections import defaultdict

from transformers import AutoModel, AutoTokenizer

import pickle

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

Google drive mount

In [2]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/your_project_folder')
torch.cuda.empty_cache()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


json메시지 로딩

In [3]:
def load_data():
    with open('processed-batch-1.json', 'r') as f:
        data = json.load(f)
    # text가 공백인 메시지는 제외
    return [msg for msg in data if msg['text'].strip()]

messages = load_data()

# 모든 메시지에 anchor_group을 미리 None으로 초기화
# 이후 앵커 트래킹 로직에서 적절히 값이 들어가도록 함
for msg in messages:
    msg['anchor_group'] = None

CHECKPOINT_FILE = "cluster_progress.json"  # 수정: 새 이름
PROCESSED_FILE = 'processed_messages.json'

EEVE 모델 로드

In [4]:
model_path = "/content/drive/MyDrive/eeve_model"

class ResourceManager:
    def __init__(self, model):
        self.model = model
    def __enter__(self):
        self.model_gpu = self.model.to('cuda')
        return self.model_gpu
    def __exit__(self, *args):
        self.model_gpu.to('cpu')
        torch.cuda.empty_cache()

class EEVEModel:
    def __init__(self, model_path):
        # 여기서 tokenizer, model 을 미리 로딩
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModel.from_pretrained(model_path).half().to('cpu')

    def generate_embedding(self, text: str) -> np.ndarray:
        # ResourceManager로 GPU 리소스를 할당받아서 임베딩 계산
        with ResourceManager(self.model) as model_gpu:
            inputs = self.tokenizer(text, return_tensors="pt",
                                    truncation=True,
                                    max_length=512).to('cuda')
            with torch.no_grad():
                outputs = model_gpu(**inputs)
            emb = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        return emb

BERTopic 로드

In [5]:
# 학습된 BERTopic 모델 불러오기 (Google Drive에 있는 trained_topic_model.pkl 사용)
trained_topic_model = BERTopic.load("trained_topic_model.pkl")

# HybridSimilarity에서 사용할 BERTopic 모델 래퍼 클래스
# transform([text], embeddings=[...])를 호출할 수 있도록 그대로 사용합니다.
class LoadedBERTopic:
    def transform(self, texts, embeddings=None):
        # embeddings 인자를 전달하면 trained_topic_model.transform()에 함께 전달합니다.
        # 모델의 transform()은 (topics, topic_dists)를 반환합니다.
        return trained_topic_model.transform(texts, embeddings=embeddings)
class LoadBERTopic:
    def transform(self, texts):
        """
        실제 BERTopic의 transform(texts) 결과로부터
        topic id나 topic 분포를 얻어서 유사도 계산에 사용.
        여기서는 단순히 0.7~0.8 사이 임의 값으로 가정
        """
        # 예시로 topic distribution을 임의 반환
        # 실제로는 bertopic_model.transform([...]) 형태로 topic 확률 벡터를 얻어야 함
        return [None, [np.array([0.7, 0.3]) for _ in texts]]

# BERTopic 내부에서 사용할 임베딩 모델도 동일하게 로딩해둬야 함 (예: sentence-transformers)
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

mention(호출) 검사 함수

In [6]:
def extract_mentions(text):
    """
    '~님' 형태의 호출 패턴을 단순 정규식으로 추출
    """
    found = re.findall(r"([\w-]+)님", text)
    return set(found)

Hybrid Topic Similarity (EEVE+BERTopic)

In [7]:
class HybridSimilarity:
    def __init__(self, eeve_model, bertopic_model, weight_eeve=0.5):
        self.eeve_model = eeve_model
        self.bertopic_model = bertopic_model  # LoadedBERTopic 인스턴스 전달
        self.weight_eeve = weight_eeve

    def cosine_sim(self, v1, v2):
        return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-12)

    def similarity(self, text, group_repr):
        # EEVE 임베딩 유사도 계산
        emb_text = self.eeve_model.generate_embedding(text)
        sim_eeve = self.cosine_sim(emb_text, group_repr["embedding"])

        # BERTopic의 transform()으로 토픽 분포 계산
        # embeddings 인자는 numpy array로 shape (1, vector_dim)이어야 함
        emb_array = np.array([emb_text])
        _, topic_dists = self.bertopic_model.transform([text], embeddings=emb_array)
        sim_bertopic = self.cosine_sim(topic_dists[0], group_repr["topic_dist"])

        hybrid_sim = self.weight_eeve * sim_eeve + (1 - self.weight_eeve) * sim_bertopic
        return hybrid_sim

앵커 트래킹 클래스

In [8]:
class AnchorTracker:
    def __init__(self, eeve_path, threshold=0.5, weight_mention=0.8, time_threshold=86400):
        self.eeve_model = EEVEModel(eeve_path)
        self.bertopic_model = LoadedBERTopic()  # 실제 BERTopic 모델 인스턴스
        self.sim_model = HybridSimilarity(self.eeve_model, self.bertopic_model, weight_eeve=0.5)
        self.groups = {}  # 그룹별 정보 저장
        self.current_id = 1
        self.threshold = threshold
        self.weight_mention = weight_mention
        self.time_threshold = time_threshold  # 추가

    def context_mention_score(self, new_text, new_sender, tail_sender):
        score = 0.0
        mentions = extract_mentions(new_text)
        if tail_sender in mentions:
            score = 1.0
        return score

    def assign_group(self, msg_text, msg_sender):
        best_gid = None
        best_score = -1.0

        for gid, info in self.groups.items():
            base_sim = self.sim_model.similarity(msg_text, info)
            mention_sc = self.context_mention_score(msg_text, msg_sender, info["tail_sender"])
            # 최종 점수: mention 점수에 weight_mention 비중 부여
            total_score = mention_sc * self.weight_mention + base_sim * (1 - self.weight_mention)
            if total_score > best_score:
                best_score = total_score
                best_gid = gid

        if best_gid is not None and best_score >= self.threshold:
            # 기존 그룹 업데이트: 새로운 임베딩과 토픽 분포 업데이트
            emb_new = self.eeve_model.generate_embedding(msg_text)
            self.groups[best_gid]["embedding"] = 0.7 * self.groups[best_gid]["embedding"] + 0.3 * emb_new
            self.groups[best_gid]["tail_sender"] = msg_sender
            self.groups[best_gid]["tail_text"] = msg_text

            emb_array = np.array([emb_new])
            _, topic_dists = self.bertopic_model.transform([msg_text], embeddings=emb_array)
            self.groups[best_gid]["topic_dist"] = 0.7 * self.groups[best_gid]["topic_dist"] + 0.3 * topic_dists[0]
            return best_gid
        else:
            # 새 그룹 생성
            new_id = self.current_id
            self.current_id += 1
            emb_new = self.eeve_model.generate_embedding(msg_text)
            emb_array = np.array([emb_new])
            _, topic_dists = self.bertopic_model.transform([msg_text], embeddings=emb_array)
            self.groups[new_id] = {
                "embedding": emb_new,
                "topic_dist": topic_dists[0],
                "tail_sender": msg_sender,
                "tail_text": msg_text
            }
            return new_id

체크포인트 메시지 로딩

In [9]:
# 메시지 로딩
if os.path.exists("processed-batch-1.json"):
    with open("processed-batch-1.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    # text가 공백인 메시지는 제외(예시)
    messages = [msg for msg in data if msg["text"].strip()]
else:
    messages = []

# 모든 메시지 anchor_group 초기화
for msg in messages:
    if "anchor_group" not in msg:
        msg["anchor_group"] = None

# 기존 진행상황 로드
if os.path.exists(CHECKPOINT_FILE):
    with open(CHECKPOINT_FILE, 'r') as f:
        progress = json.load(f)
else:
    progress = {"processed_idx": 0, "failed_ids": []}

tracker = AnchorTracker(
    eeve_path="/content/drive/MyDrive/eeve_model",
    threshold=0.5,
    weight_mention=0.8,
    time_threshold=86400
)


임베딩 & 앵커 트래킹

In [ ]:
for i in tqdm(range(progress["processed_idx"], len(messages)), desc="임베딩 생성"):
    try:
        # 이미 임베딩과 anchor_group 있으면 넘어감
        if 'embedding' in messages[i] and messages[i].get('anchor_group') not in [None, 0]:
            continue

        # 임베딩 생성
        emb = tracker.eeve_model.generate_embedding(messages[i]['text']).tolist()
        messages[i]['embedding'] = emb

        # 앵커 그룹 할당
        # embedding_np와 msg_time 인자는 더 이상 전달하지 않습니다.
        sender = messages[i]['sender']
        text   = messages[i]['text']
        assigned_gid = tracker.assign_group(
            msg_text=text,
            msg_sender=sender
        )
        messages[i]['anchor_group'] = assigned_gid

        progress["processed_idx"] = i + 1

        # 10개 단위로 임시 저장
        if (i + 1) % 10 == 0:
            with open(CHECKPOINT_FILE, 'w') as f:
                json.dump(progress, f)
            with open(PROCESSED_FILE, 'w', encoding='utf-8') as f:
                json.dump(messages, f, ensure_ascii=False, indent=4)

    except Exception as e:
        print(f"에러 @ {i}: {str(e)}")
        progress["failed_ids"].append(i)
        continue

with open(CHECKPOINT_FILE, 'w') as f:
    json.dump(progress, f)
with open(PROCESSED_FILE, 'w', encoding='utf-8') as f:
    json.dump(messages, f, ensure_ascii=False, indent=4)


임베딩 생성:  54%|█████▍    | 142/263 [51:16<1:28:36, 43.94s/it]

anchor group merge

In [ ]:
def compute_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1)*np.linalg.norm(vec2))

def merge_anchor_groups(anchor_groups, context_threshold=3600, topic_threshold=0.8):
    """
    anchor_groups 형식 (예시):
    {
      group_id: {
        "messages": [ (text, timestamp, embedding), ... ],
        "group_embedding": np.array([...])     # 그룹 전체를 대표하는 임베딩
      },
      ...
    }
    """

    group_list = list(anchor_groups.items())  # (group_id, group_info) 튜플 목록
    merged = {}

    for gid, info in group_list:
        # 이미 merge로 흡수된 그룹 제외
        if gid in merged:
            continue
        if gid not in merged:
            merged[gid] = info

        # tail 메시지(가장 마지막 메시지) 조회
        tail_msg = sorted(info["messages"], key=lambda x: x[1])[-1]
        tail_time = tail_msg[1]
        tail_emb = tail_msg[2]

        # 나머지 그룹들과 비교
        for other_gid, other_info in group_list:
            if other_gid == gid or other_gid in merged and merged[other_gid] is None:
                continue

            # 다른 그룹 tail
            other_tail = sorted(other_info["messages"], key=lambda x: x[1])[-1]
            other_tail_time = other_tail[1]
            other_tail_emb = other_tail[2]

            # 맥락(시간) 연속성 평가: 두 tail 메시지 시간 차 체크
            time_diff = abs(tail_time - other_tail_time).total_seconds()
            if time_diff > context_threshold:
                continue

            # 토픽(주제) 유사도 평가
            sim = compute_similarity(info["group_embedding"], other_info["group_embedding"])
            if sim >= topic_threshold:
                # 병합 처리
                merged[gid]["messages"].extend(other_info["messages"])
                # 그룹 전체 임베딩 업데이트(간단히 평균)
                updated_emb = 0.5 * (merged[gid]["group_embedding"] + other_info["group_embedding"])
                merged[gid]["group_embedding"] = updated_emb
                # other_gid를 소멸 처리
                merged[other_gid] = None

    # 최종적으로 None 아닌 그룹만 추려서 반환
    final = {}
    for k, v in merged.items():
        if v is not None:
            final[k] = v
    return final


최종 결과 저장

In [ ]:
group_dict = defaultdict(list)
for msg in messages:
    # 여기서 KeyError가 발생하지 않도록 anchor_group이 None이 아닌지 확인
    if msg['anchor_group'] is None:
        msg['anchor_group'] = -1  # 혹은 기타 임시값
    group_dict[msg['anchor_group']].append(msg['text'])

with open('grouped_messages.json', 'w', encoding='utf-8') as f:
    json.dump(group_dict, f, ensure_ascii=False, indent=4)
